In [1]:
import matplotlib.pyplot as plt
import astropy.table as table
import astropy.stats as stats
import numpy as np

from newport import *
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
import astropy.units as u

In [2]:
# List of RA and Dec
ras = [149.12329167, 149.11337083, 149.1408375, 149.1254875, 149.14237917, 149.09957917, 149.09925833, 149.14972083, 149.134125, 149.0973375, 149.09987083, 149.0956875, 149.1593, 149.0904375, 149.16145417, 149.10824583, 149.12028333, 149.07483638, 149.07452917, 149.07863696, 149.10703785, 149.14167738, 149.19347924, 149.18083805, 149.05402844, 149.06473608, 149.17446644]
decs = [-24.09912861, -24.10344694, -24.10548028, -24.08288778, -24.09318361, -24.10292639, -24.09616278, -24.09940139, -24.12276083, -24.09103194, -24.11740111, -24.08020972, -24.1150075, -24.07894889, -24.08054722, -24.13615611, -24.05940444, -24.06832958, -24.0809112, -24.11477845, -24.14304355, -24.05236132, -24.08765455, -24.1007293, -24.09474303, -24.11812912, -24.04326733]

# Create SkyCoord objects
coords = SkyCoord(ra=ras*u.degree, dec=decs*u.degree, frame='icrs')

# Query Gaia for each coordinate
results = []
for coord in coords:
    query = f"""
    SELECT source_id, ra, dec
    FROM gaiadr3.gaia_source
    WHERE 1=CONTAINS(
        POINT('ICRS', ra, dec),
        CIRCLE('ICRS', {coord.ra.degree}, {coord.dec.degree}, 0.001))
    """
    job = Gaia.launch_job_async(query)
    result = job.get_results()
    results.append(result['SOURCE_ID'][0])

INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


In [4]:
N_COL_HEAD = 5
fn = 'HD_86226'

In [8]:
phot_table_all_band = table.Table.read(f'tables/phot/photometry_{fn}.fits')

for band in ['B', 'V', 'R', 'I']:
    phot_table = phot_table_all_band[phot_table_all_band['band'] == band]

    if len(phot_table) < 1:
        print(f'{fn}\t{band}\t skipped')
        continue

    valid_colnames = []
    for r in results:
        r = str(r)
        if r in phot_table.colnames:
            valid_colnames.append(r)

    print(f'{fn}\t{band}\t{len(valid_colnames)}')  # DEV

    valid_table = phot_table[valid_colnames]

    relphot_table = table.Table(names=valid_table.colnames)
    for row in valid_table:
        # relphot_row = []
        row_as_array = np.array(list(row))
        row_sum = np.nansum(row_as_array)
        relphot_table.add_row(row_as_array / (row_sum - row_as_array))  # TODO
        # for i, value in enumerate(row):
        #     relphot_row.append(value / (row_sum - value))
        # relphot_table_no_head.add_row(relphot_row)

    # relphot_table = table.hstack([phot_table[phot_table.colnames[ : N_COL_HEAD]], relphot_table_no_head])
    relphot_table = table.hstack([phot_table[phot_table.colnames[: N_COL_HEAD]], relphot_table])

    gpb = relphot_table.group_by(['night', 'band'])
    binned_relphot = gpb.groups.aggregate(np.median)
    # TODO add std or mad_std here and save it

    # band-night normalization
    for colname in binned_relphot.colnames:
        if colname.isdigit():
            med = np.nanmedian(binned_relphot[colname])
            binned_relphot[colname] /= med
            relphot_table[colname] /= med

    binned_relphot.write(f'tables/binned/binned_{fn}_{band}_ap.fits', overwrite=True)
    relphot_table.write(f'tables/relphot/relphot_{fn}_{band}_ap.fits', overwrite=True)

HD_86226	B	12


/var/folders/zc/6fl6c21x1558785lrq_dry700000gn/T/ipykernel_1520/3049883249.py:23: UserWarning: Warning: converting a masked element to nan.
  row_as_array = np.array(list(row))


HD_86226	V	12


HD_86226	R	12


/var/folders/zc/6fl6c21x1558785lrq_dry700000gn/T/ipykernel_1520/3049883249.py:23: UserWarning: Warning: converting a masked element to nan.
  row_as_array = np.array(list(row))


HD_86226	I	12


/var/folders/zc/6fl6c21x1558785lrq_dry700000gn/T/ipykernel_1520/3049883249.py:23: UserWarning: Warning: converting a masked element to nan.
  row_as_array = np.array(list(row))


band,night,jd,airmass,5660492297395345408,5660492331755081472,5660494943095202176,5660495217973451520,5660495011814679296,5660494805656252928,5660491575840846592,5660495488555204992,5660515662017429120,5660495076238345728,5660494771296515840,5660516035678748544
bytes32,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
I,20221224,2459938.862461227,2.43021181555,0.8399959839709917,1.2035289742791617,1.1677872962352147,1.2836881895256664,1.2978563091412325,1.2030290097061382,1.1128816184265151,1.1914737010731093,1.2674067646011018,1.165183269756166,1.0236754230293144,1.1216887786782002
I,20221225,2459939.8835711805,2.502264598,0.9783762513345725,1.023321706950532,1.2408198077208312,1.118714091281833,0.7999014161873431,1.0206649199584308,0.9449782420787834,0.879972130718996,1.0978977572646706,0.8398211749825583,0.9010310240841524,1.1626481232845525
I,20230117,2459962.809502315,2.45632195722,0.769946588474634,1.3087999208126728,1.380342289186594,1.3265217879468216,1.328092818383269,1.3298268627698637,1.2729764736098463,1.2629313635077115,1.2434501709782464,1.3582608598654997,1.3025271918000412,1.3757651502768438
I,20230118,2459963.8102238425,2.46830335605,0.9061527349389126,1.1529954274706404,1.2526519721997025,1.1172818085318736,1.2246333099152804,1.1352965123025252,1.1101910080516326,1.0011238096842832,1.1171104000304295,0.9804247093803328,1.068771496479973,1.170912792668273
I,20230213,2459989.7600385416,2.58300433556,0.7713288267124339,1.2740267621425525,1.2832027279109475,1.3658396916575204,1.1621710814087154,1.2847995698145802,1.3494521755279931,0.8905443505540614,1.1933027681901787,1.5114020887372686,1.343480492674571,1.185423607538117
I,20230218,2459994.7263988424,2.46919666713,0.7802993260006851,1.2947469064750616,1.2921518653207742,1.1644429966333154,1.216530975866266,1.2722307734894271,1.30351360590816,1.2551294146680947,1.3806643784686712,1.265445718733042,1.1770174500105857,1.2884294732466552
I,20230311,2460015.6813616897,2.5295173808,0.8888227104026736,1.1367829181792772,1.1651410489100344,1.053406819852878,0.9719269949232177,1.132229415614847,1.1276972131933092,1.1759449904291548,1.2622410429499387,1.0305030513043134,1.0364867027642894,1.0487193177856253
I,20230319,2460023.6782881366,2.68246285723,nan,1.7956003406066787,1.7995540784151438,1.8403445500002065,1.7262452396305508,1.7756705426302146,1.8443831360207283,1.7433705092921148,1.660535858501878,1.774466962514465,1.8780322772389568,1.748520238327501
I,20230320,2460024.615848611,2.427573905995,1.0138067191889029,0.9612302086323447,0.8768278710055251,0.9759881982779902,0.9379382970473661,1.0048768639851986,0.9403628532145745,0.963335848740315,1.03816733430518,1.0196092834697301,1.0470356669685035,1.0031495318223205


In [13]:
query = f"""
SELECT *
FROM gaiadr3.vari_summary
WHERE source_id = {5157183324996790272}
"""

# Execute the query
job = Gaia.launch_job_async(query)
result = job.get_results()


INFO: Query finished. [astroquery.utils.tap.core]


In [14]:
result

solution_id,SOURCE_ID,num_selected_g_fov,mean_obs_time_g_fov,time_duration_g_fov,min_mag_g_fov,max_mag_g_fov,mean_mag_g_fov,median_mag_g_fov,range_mag_g_fov,trimmed_range_mag_g_fov,std_dev_mag_g_fov,skewness_mag_g_fov,kurtosis_mag_g_fov,mad_mag_g_fov,abbe_mag_g_fov,iqr_mag_g_fov,stetson_mag_g_fov,std_dev_over_rms_err_mag_g_fov,outlier_median_g_fov,num_selected_bp,mean_obs_time_bp,time_duration_bp,min_mag_bp,max_mag_bp,mean_mag_bp,median_mag_bp,range_mag_bp,trimmed_range_mag_bp,std_dev_mag_bp,skewness_mag_bp,kurtosis_mag_bp,mad_mag_bp,abbe_mag_bp,iqr_mag_bp,stetson_mag_bp,std_dev_over_rms_err_mag_bp,outlier_median_bp,num_selected_rp,mean_obs_time_rp,time_duration_rp,min_mag_rp,max_mag_rp,mean_mag_rp,median_mag_rp,range_mag_rp,trimmed_range_mag_rp,std_dev_mag_rp,skewness_mag_rp,kurtosis_mag_rp,mad_mag_rp,abbe_mag_rp,iqr_mag_rp,stetson_mag_rp,std_dev_over_rms_err_mag_rp,outlier_median_rp,in_vari_classification_result,in_vari_rrlyrae,in_vari_cepheid,in_vari_planetary_transit,in_vari_short_timescale,in_vari_long_period_variable,in_vari_eclipsing_binary,in_vari_rotation_modulation,in_vari_ms_oscillator,in_vari_agn,in_vari_microlensing,in_vari_compact_companion
,,,d,d,mag,mag,mag,mag,mag,mag,mag,,,mag,,mag,,,,,d,d,mag,mag,mag,mag,mag,mag,mag,,,mag,,mag,,,,,d,d,mag,mag,mag,mag,mag,mag,mag,,,mag,,mag,,,,,,,,,,,,,,,
int64,int64,int16,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
375316653866487564,5157183324996790272,48,2271.604855050926,906.34644,12.069946,12.1093025,12.088745,12.090389,0.03935684,0.035218652,0.010258049,-0.017353622,-0.5266662,0.009013334,0.5643097,0.012692366,4.2080665,4.316349,18.95803,46,2254.6179482451603,906.52264,13.314403,13.367763,13.343605,13.346238,0.053359915,0.03664869,0.012523753,-0.5399279,-0.554474,0.012731082,0.5766581,0.020901715,4.271479,5.8763566,14.919454,47,2256.556867367543,906.52264,10.96033,10.985793,10.974807,10.975503,0.025463788,0.01997067,0.0057589356,-0.24099652,-0.1910345,0.0055653397,0.7493987,0.008700385,4.431943,6.2637463,16.560022,True,False,False,False,False,False,False,False,False,False,False,False
